In [20]:
import pandas as pd
from collections import defaultdict
import joblib

# Load dataset and saved artifacts
df = pd.read_csv("crop_yield.csv")
best_model = joblib.load("crop_yield_model.joblib")
encoder = joblib.load("onehot_encoder.joblib")

categorical = ['Crop', 'Season', 'State']

def recommend_and_print_crops(state_name, annual_rainfall_input, model, df, encoder):
    # Convert state name to proper case
    state_name = state_name.title()
    state_df = df[df['State'] == state_name]
    unique_crops = state_df['Crop'].unique()
    raw_recommendations = defaultdict(list)

    for crop in unique_crops:
        crop_data = state_df[state_df['Crop'] == crop]
        avg_features = crop_data[['Crop_Year', 'Area', 'Production', 'Fertilizer', 'Pesticide']].mean()

        # Weighted annual rainfall (weight = 2)
        weighted_rainfall = annual_rainfall_input * 2

        input_df = pd.DataFrame({
            'Crop': [crop],
            'Season': [crop_data['Season'].mode()[0] if not crop_data['Season'].mode().empty else 'Unknown'],
            'State': [state_name]
        })
        season = input_df['Season'][0]

        encoded_cat = encoder.transform(input_df[categorical])
        encoded_df = pd.DataFrame(encoded_cat, columns=encoder.get_feature_names_out())

        numeric_values = avg_features.values.tolist() + [weighted_rainfall]
        numeric_columns = list(avg_features.index) + ['Annual_Rainfall_Weighted']

        numeric_df = pd.DataFrame([numeric_values], columns=numeric_columns)

        X_pred = pd.concat([encoded_df.reset_index(drop=True), numeric_df.reset_index(drop=True)], axis=1)
        X_pred = X_pred.reindex(columns=model.feature_names_in_, fill_value=0)

        predicted_yield = model.predict(X_pred)[0]
        raw_recommendations[season].append((crop, predicted_yield))

    # Scale predicted yields to 1-100
    all_scores = [score for crops in raw_recommendations.values() for _, score in crops]
    min_score = min(all_scores)
    max_score = max(all_scores)

    def scale(score):
        if max_score == min_score:
            return 100
        return 1 + 99 * (score - min_score) / (max_score - min_score)

    scaled_recommendations = defaultdict(list)
    for season, crops in raw_recommendations.items():
        for crop, score in crops:
            scaled_score = round(scale(score), 2)
            scaled_recommendations[season].append((crop, scaled_score, round(score, 2)))
        scaled_recommendations[season].sort(key=lambda x: x[1], reverse=True)

    # Print output grouped by season with score and estimated yield
    print(f"\nTop crops recommended for {state_name} with annual rainfall {annual_rainfall_input} grouped by season:")
    for season, crops in scaled_recommendations.items():
        print(f"\nSeason: {season}")
        print(f"{'Crop':<20} {'Score':>6} {'Est. Yield':>12}")
        for crop, score, est_yield in crops:
            print(f"{crop:<20} {score:>6} {est_yield:>12}")

# Example usage:
state_input = input("Enter state name: ")
rainfall_input = float(input("Enter annual rainfall (e.g., in mm): "))

recommend_and_print_crops(state_input, rainfall_input, best_model, df, encoder)



Top crops recommended for Assam with annual rainfall 1234.0 grouped by season:

Season: Whole Year 
Crop                  Score   Est. Yield
Banana                100.0        18.36
Ginger                58.53        10.94
Potato                54.92         10.3
Tapioca               48.45         9.14
Sweet potato           32.8         6.35
Onion                 27.85         5.46
Turmeric               9.19         2.13
Black pepper           3.62         1.13
Dry chillies           2.86         0.99
Tobacco                2.43         0.92
Arecanut               2.05         0.85

Season: Kharif     
Crop                  Score   Est. Yield
Jute                  53.89        10.11
Mesta                 14.91         3.15
Maize                  3.59         1.12
Arhar/Tur              1.81         0.81
Cotton(lint)           1.61         0.77
Small millets          1.58         0.77
Castor seed            1.54         0.76
Sesamum                 1.5         0.75
Niger seed       